# Library Imports

In [1]:
# import necessary libraries
import pandas as pd
import numpy as np
from random import sample
from collections import Counter
from typing import List, Dict, Union

# Parsing and Cleaning Data

In [2]:
file_name = "SpotifyFeatures.csv"
spotify_data = pd.read_csv(file_name)
spotify_data.dropna()
spotify_data.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390


In [3]:
def get_counter_from_column(dataframe: pd.DataFrame, column: str) -> Counter:
    unique = Counter(dataframe[column])
    print(f"Unique values for {column}: {dict(unique)}")
    return unique

# getting an idea of what non-numerical values we might have for data so we can map them to numbers later
# excluding things such as artist_name, track_name, and track_id due to the sheer size and number of unique elements
column_names = ["genre", "key", "mode", "time_signature"]
unique_genres, unique_keys, unique_modes, unique_time_signatures = [get_counter_from_column(spotify_data, column) for column in column_names]

# 0/4 time signature seems to be an outlier value for time signature, so we'll likely want to remove them
zero_four_time_signature = spotify_data[spotify_data["time_signature"] == "0/4"]
print(f"Number of tracks with 0/4 time signature: {len(zero_four_time_signature)}")

Unique values for genre: {'Movie': 7806, 'R&B': 8992, 'A Capella': 119, 'Alternative': 9263, 'Country': 8664, 'Dance': 8701, 'Electronic': 9377, 'Anime': 8936, 'Folk': 9299, 'Blues': 9023, 'Opera': 8280, 'Hip-Hop': 9295, "Children's Music": 5403, 'Children’s Music': 9353, 'Rap': 9232, 'Indie': 9543, 'Classical': 9256, 'Pop': 9386, 'Reggae': 8771, 'Reggaeton': 8927, 'Jazz': 9441, 'Rock': 9272, 'Ska': 8874, 'Comedy': 9681, 'Soul': 9089, 'Soundtrack': 9646, 'World': 9096}
Unique values for key: {'C#': 23201, 'F#': 15222, 'C': 27583, 'F': 20279, 'G': 26390, 'E': 17390, 'D#': 7566, 'G#': 15159, 'D': 24077, 'A#': 15526, 'A': 22671, 'B': 17661}
Unique values for mode: {'Major': 151744, 'Minor': 80981}
Unique values for time_signature: {'4/4': 200760, '5/4': 5238, '3/4': 24111, '1/4': 2608, '0/4': 8}
Number of tracks with 0/4 time signature: 8


In [4]:
def remove_time_signature(dataframe: pd.DataFrame, time_signature: str) -> pd.DataFrame:
    return dataframe[dataframe["time_signature"] != time_signature]

def create_integer_mapping(items: List[str]) -> Dict[str, int]:
    return {item: i for i, item in enumerate(items)}

# remove rows that have a 0/4 time signature 
spotify_data = remove_time_signature(spotify_data, "0/4")
zero_four_time_signature = spotify_data[spotify_data["time_signature"] == "0/4"]
print(f"Number of tracks with 0/4 time signature: {len(zero_four_time_signature)}")

genre_integer_mapping = create_integer_mapping(list(unique_genres))
key_integer_mapping = create_integer_mapping(list(unique_keys))
mode_integer_mapping = create_integer_mapping(list(unique_modes))
time_signature_mapping = create_integer_mapping(list(unique_time_signatures))

print(f"Genre to integer mapping: {genre_integer_mapping}")
print(f"Key to integer mapping: {key_integer_mapping}")
print(f"Mode to integer mapping: {mode_integer_mapping}")
print(f"Time signature to integer mapping: {time_signature_mapping}")

Number of tracks with 0/4 time signature: 0
Genre to integer mapping: {'Movie': 0, 'R&B': 1, 'A Capella': 2, 'Alternative': 3, 'Country': 4, 'Dance': 5, 'Electronic': 6, 'Anime': 7, 'Folk': 8, 'Blues': 9, 'Opera': 10, 'Hip-Hop': 11, "Children's Music": 12, 'Children’s Music': 13, 'Rap': 14, 'Indie': 15, 'Classical': 16, 'Pop': 17, 'Reggae': 18, 'Reggaeton': 19, 'Jazz': 20, 'Rock': 21, 'Ska': 22, 'Comedy': 23, 'Soul': 24, 'Soundtrack': 25, 'World': 26}
Key to integer mapping: {'C#': 0, 'F#': 1, 'C': 2, 'F': 3, 'G': 4, 'E': 5, 'D#': 6, 'G#': 7, 'D': 8, 'A#': 9, 'A': 10, 'B': 11}
Mode to integer mapping: {'Major': 0, 'Minor': 1}
Time signature to integer mapping: {'4/4': 0, '5/4': 1, '3/4': 2, '1/4': 3, '0/4': 4}


In [5]:
# replace values based on a list of dictionary mappings
def replace_mappings(dataframe: pd.DataFrame, mappings: List[Dict[str, int]]) -> pd.DataFrame:
    merged_mapping = {}
    new_dataframe = dataframe.copy()
    for mapping in mappings:
        merged_mapping.update(mapping)
    new_dataframe = new_dataframe.replace(to_replace=merged_mapping)
    return new_dataframe

# use the mappings from the previous cell and show that the only rows with a non-integer/float datatype are artist_name, track_name, and track_id
spotify_data_replaced = replace_mappings(spotify_data, [genre_integer_mapping, key_integer_mapping, mode_integer_mapping, time_signature_mapping])
spotify_data_replaced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 232717 entries, 0 to 232724
Data columns (total 18 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   genre             232717 non-null  int64  
 1   artist_name       232717 non-null  object 
 2   track_name        232717 non-null  object 
 3   track_id          232717 non-null  object 
 4   popularity        232717 non-null  int64  
 5   acousticness      232717 non-null  float64
 6   danceability      232717 non-null  float64
 7   duration_ms       232717 non-null  int64  
 8   energy            232717 non-null  float64
 9   instrumentalness  232717 non-null  float64
 10  key               232717 non-null  int64  
 11  liveness          232717 non-null  float64
 12  loudness          232717 non-null  float64
 13  mode              232717 non-null  int64  
 14  speechiness       232717 non-null  float64
 15  tempo             232717 non-null  float64
 16  time_signature    23

In [6]:
# drop a given list of column names from a dataframe
def drop_columns(dataframe: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
    new_dataframe = dataframe.copy()
    for column in columns:
        new_dataframe = new_dataframe.drop(column, 1)
    return new_dataframe

def print_random_rows(array: Union[np.ndarray, pd.Series], num: int) -> None:
    indexes = sample(range(len(array)), num)
    for index in indexes:
        item = array[index]
        print(f"\t{item}, {item.ndim} dimensions")

random_samples = 3

# create our array of target variables
Y = spotify_data_replaced["genre"]
print(f"{random_samples} random samples from Y, {Y.ndim} dimensional:")
print_random_rows(Y, random_samples)

# drop genres, artist names, track names, and track ids from dataframe
spotify_data_dropped = drop_columns(spotify_data_replaced, ["genre", "artist_name", "track_name", "track_id"])

# create our array of samples and variables
X = spotify_data_dropped.to_numpy()
print(f"{random_samples} random samples from X, {X.ndim} dimensional:")
print_random_rows(X, random_samples)

3 random samples from Y, 1 dimensional:
	9, 0 dimensions
	5, 0 dimensions
	6, 0 dimensions
3 random samples from X, 2 dimensional:
	[ 1.30000e+01  8.99000e-01  2.88000e-01  3.44956e+05  1.70000e-01
  7.10000e-01  2.00000e+00  6.16000e-02 -1.56110e+01  1.00000e+00
  4.25000e-02  1.34469e+02  0.00000e+00  1.33000e-01], 1 dimensions
	[ 9.00000e+00  9.23000e-01  3.02000e-01  1.80373e+05  7.87000e-02
  3.24000e-03  8.00000e+00  2.87000e-01 -2.60600e+01  0.00000e+00
  4.34000e-02  5.60170e+01  2.00000e+00  1.75000e-01], 1 dimensions
	[ 2.80000e+01  9.91000e-01  3.98000e-01  2.16586e+05  2.07000e-01
  1.07000e-02  1.00000e+01  1.28000e-01 -1.19340e+01  0.00000e+00
  4.03000e-02  1.18590e+02  0.00000e+00  2.91000e-01], 1 dimensions


# 